In [ ]:
from contextlib import contextmanager
import ur
from ..process import run
from . import branch as _branch

class Checkout:
    def __call__(checkout, branch, ref=None, create=False, exist=False, ctx=True, sha_ok=True):
        prev = _branch.current(sha_ok=sha_ok)
        if exist or not create:
            run('git','checkout',branch,ref)
        else:
            run('git','checkout','-B',branch,ref)
        
        if ctx:
            @contextmanager
            def ret():
                try:
                    yield prev
                finally:
                    checkout(prev, exist=True, ctx=False)
            
            return ret()
        else:
            return prev